# Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

 # Libraries

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GINConv,GINEConv,PDNConv
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_geometric.utils.convert import from_scipy_sparse_matrix

import csv
import time
import numpy as np
import scipy.sparse as sp
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm

import torch.nn as nn
from torch import optim
import os

# Loading data

In [ ]:
PATH = "drive/MyDrive/Altegra_data"

def load_data(new_edge_loading=False): 
    """
    Function that loads graphs
    """  
    graph_indicator = np.loadtxt(os.path.join(PATH,"graph_indicator.txt"), dtype=np.int64)
    _,graph_size = np.unique(graph_indicator, return_counts=True)
    
    edges = np.loadtxt(os.path.join(PATH,"edgelist.txt"), dtype=np.int64, delimiter=",")
    edges_inv = np.vstack((edges[:,1], edges[:,0]))
    edges = np.vstack((edges, edges_inv.T))
    s = edges[:,0]*graph_indicator.size + edges[:,1]
    idx_sort = np.argsort(s)
    edges = edges[idx_sort,:]
    edges,idx_unique =  np.unique(edges, axis=0, return_index=True)
    A = sp.csr_matrix((np.ones(edges.shape[0]), (edges[:,0], edges[:,1])), shape=(graph_indicator.size, graph_indicator.size))
    
    
    x = np.loadtxt(os.path.join(PATH,"node_attributes.txt"), delimiter=",")
    edge_attr = np.loadtxt(os.path.join(PATH,"edge_attributes.txt"), delimiter=",")
    edge_attr = np.vstack((edge_attr,edge_attr))
    edge_attr = edge_attr[idx_sort,:]
    edge_attr = edge_attr[idx_unique,:]
    
    adj = []
    features = []
    edge_features = []
    idx_n = 0
    idx_m = 0
    for i in range(graph_size.size):
        adj.append(A[idx_n:idx_n+graph_size[i],idx_n:idx_n+graph_size[i]])
        edge_features.append(edge_attr[idx_m:idx_m+adj[i].nnz,:])
        features.append(x[idx_n:idx_n+graph_size[i],:])
        idx_n += graph_size[i]
        idx_m += adj[i].nnz

    return adj, features, edge_features

def normalize_adjacency(A):
    """
    Function that normalizes an adjacency matrix
    """
    n = A.shape[0]
    A += sp.identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D = sp.diags(inv_degs)
    A_normalized = D.dot(A)

    return A_normalized

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """
    Function that converts a Scipy sparse matrix to a sparse Torch tensor
    """
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# Models

In [ ]:
# from https://mlabonne.github.io/blog/gin/
class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.lin1 = Linear(hidden_dim*3, hidden_dim*3)
        self.lin2 = Linear(hidden_dim*3, n_class)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, batch):
        # Node embeddings 
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [ ]:
class GINE(torch.nn.Module):
    """GINE"""
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super(GINE, self).__init__()
        self.conv1 = GINEConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        self.conv2 = GINEConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        self.conv3 = GINEConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        
        self.conv4 = GINEConv(
            Sequential(Linear(hidden_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        self.conv5 = GINEConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        self.conv6 = GINEConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()),edge_dim=5)
        self.lin1 = Linear(hidden_dim*3, hidden_dim*3)
        self.lin2 = Linear(hidden_dim*3, n_class)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_features, batch):

        h1 = self.conv1(x, edge_index, edge_features)
        h2 = self.conv2(h1, edge_index, edge_features)
        h3 = self.conv3(h2, edge_index, edge_features)

        # # Node embeddings 
        h1 = self.conv4(h1, edge_index, edge_features)
        h2 = self.conv5(h2, edge_index, edge_features)
        h3 = self.conv6(h3, edge_index, edge_features)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [ ]:
# from https://mlabonne.github.io/blog/gin/
class GIN2Graphs(torch.nn.Module):
    """GIN2Graphs"""
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super(GIN2Graphs, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        

        self.conv4 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv5 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv6 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        
        self.lin1 = Linear(hidden_dim*6, hidden_dim*6)
        self.lin2 = Linear(hidden_dim*6, n_class)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index_distance_based,edge_index_bond_based, batch):
        # First Graph info
        h1 = self.conv1(x, edge_index_distance_based)
        h2 = self.conv2(h1, edge_index_distance_based)
        h3 = self.conv3(h2, edge_index_distance_based)

        h4 = self.conv4(x, edge_index_bond_based)
        h5 = self.conv5(h4, edge_index_bond_based)
        h6 = self.conv6(h5, edge_index_bond_based)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Graph-level readout
        h4 = global_add_pool(h4, batch)
        h5 = global_add_pool(h5, batch)
        h6 = global_add_pool(h6, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3,h4,h5,h6), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [ ]:
class PDN(torch.nn.Module):
    """ Pathfinder Discovery Networks for Neural Message Passing paper"""
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super().__init__()
        self.conv1 = PDNConv(input_dim, hidden_dim, edge_dim=5, hidden_channels=2)
        self.conv2 = PDNConv(hidden_dim, hidden_dim, edge_dim=5, hidden_channels=2)
        self.conv3 = PDNConv(hidden_dim, hidden_dim, edge_dim=5, hidden_channels=2)
        self.lin1 = Linear(hidden_dim, hidden_dim)
        self.lin2 = Linear(hidden_dim, n_class)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_features, batch):
        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_features)
        h2 = self.conv2(h1, edge_index, edge_features)
        h3 = self.conv3(h2, edge_index, edge_features)

        # Graph-level readout
        h = global_add_pool(h3, batch)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

In [ ]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [ ]:
embed_dim = 16
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self,input_dim,num_classes):
        super(Net, self).__init__()

        self.conv0 = GINConv(
            Sequential(Linear(input_dim, embed_dim),
                       BatchNorm1d(embed_dim), ReLU(),
                       Linear(embed_dim, embed_dim), ReLU()))
        self.conv1 = SAGEConv(embed_dim, embed_dim)
        self.pool1 = TopKPooling(embed_dim, ratio=0.8)
        self.conv2 = SAGEConv(embed_dim, embed_dim)
        self.pool2 = TopKPooling(embed_dim, ratio=0.8)
        self.conv3 = SAGEConv(embed_dim, embed_dim)
        self.pool3 = TopKPooling(embed_dim, ratio=0.8)
        # self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(embed_dim*2, embed_dim)
        self.lin2 = torch.nn.Linear(embed_dim, embed_dim//2)
        self.lin3 = torch.nn.Linear(embed_dim//2, num_classes)
        self.bn1 = torch.nn.BatchNorm1d(embed_dim)
        self.bn2 = torch.nn.BatchNorm1d(embed_dim//2)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, x, edge_index, batch):
        # x = self.item_embedding(x)
        # x = x.squeeze(1)     
        x = F.relu(self.conv0(x, edge_index))
        x = F.relu(self.conv1(x, edge_index))

        tmp = self.pool1(x, edge_index, None, batch)
        x, edge_index, _, batch, _, _ = tmp
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _  = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = F.log_softmax(self.lin3(x), dim=1).squeeze(1)

        return x

In [ ]:
# from https://mlabonne.github.io/blog/gin/
class GIN4Graphs(torch.nn.Module):
    """GIN4Graphs"""
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super(GIN4Graphs, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        

        self.conv4 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv5 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv6 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        
        self.conv7 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv8 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv9 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        
        self.conv10 = GINConv(
            Sequential(Linear(input_dim, hidden_dim),
                       BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv11 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        self.conv12 = GINConv(
            Sequential(Linear(hidden_dim, hidden_dim), BatchNorm1d(hidden_dim), ReLU(),
                       Linear(hidden_dim, hidden_dim), ReLU()))
        
        self.lin1 = Linear(hidden_dim*3 * 4, hidden_dim*3 * 4)
        self.lin2 = Linear(hidden_dim*3 * 4, n_class)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index_type1,edge_index_type2,edge_index_type3,edge_index_type4, batch):
        # First Graph info
        h1 = self.conv1(x, edge_index_type1)
        h2 = self.conv2(h1, edge_index_type1)
        h3 = self.conv3(h2, edge_index_type1)

        h4 = self.conv4(x, edge_index_type2)
        h5 = self.conv5(h4, edge_index_type2)
        h6 = self.conv6(h5, edge_index_type2)

        h7 = self.conv7(x, edge_index_type3)
        h8 = self.conv8(h7, edge_index_type3)
        h9 = self.conv9(h8, edge_index_type3)

        h10 = self.conv10(x, edge_index_type4)
        h11 = self.conv11(h10, edge_index_type4)
        h12 = self.conv12(h11, edge_index_type4)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Graph-level readout
        h4 = global_add_pool(h4, batch)
        h5 = global_add_pool(h5, batch)
        h6 = global_add_pool(h6, batch)

        # Graph-level readout
        h7 = global_add_pool(h7, batch)
        h8 = global_add_pool(h8, batch)
        h9 = global_add_pool(h9, batch)

        # Graph-level readout
        h10 = global_add_pool(h10, batch)
        h11 = global_add_pool(h11, batch)
        h12 = global_add_pool(h12, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3,h4,h5,h6,h7, h8, h9,h10,h11,h12), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = self.dropout(h)
        h = self.lin2(h)
        
        return h, F.log_softmax(h, dim=1)

# Getting Data and Choosing Model

In [ ]:
adj, features, edge_features = load_data() 
# Normalize adjacency matrices
adj = [normalize_adjacency(A) for A in adj]

In [ ]:
adj_train_no_split = list()
features_train_no_split = list()
edges_features_train_no_split = list()
y_train_no_split = list()
adj_test = list()
features_test = list()
edges_features_test = list()
proteins_test = list()
print("Loading Data")

method_number = 2
#normalizing values
if method_number == 1:
  #Method 1
  MEAN_COORDS = np.zeros(3)
  MIN_COORDS = np.zeros(3)
  MAX_COORDS = np.zeros(3)
#Method 2
elif method_number == 2:
  from sklearn.preprocessing import RobustScaler
  transformer = RobustScaler()
  transformer_amino = RobustScaler()
  transformer_dist = RobustScaler()
  all_coords = []
  all_features = []
  all_dist = []
# IMPORTANT: Only normalize coordinates (0-2) and amino acid features (26-86)
with open(os.path.join(PATH,'graph_labels.txt'), 'r') as f:
    for i,line in tqdm(enumerate(f)):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            proteins_test.append(t[0])
            adj_test.append(adj[i])
            features_test.append(features[i])
            edges_features_test.append(edge_features[i])
        else:
            adj_train_no_split.append(adj[i])
            feat = features[i]
            efeat = edge_features[i]
            #Method 1
            if method_number == 1:
              MEAN_COORDS += np.mean(feat[:,0:3],axis=0)
              MIN_COORDS = np.min([MIN_COORDS,np.min(feat[:,0:3],axis=0)],axis=0)
              MAX_COORDS = np.max([MAX_COORDS,np.max(feat[:,0:3],axis=0)],axis=0)
            #Method 2
            elif method_number == 2:
              all_coords.append(feat[:,0:3])
              all_features.append(feat[:,25:])
              all_dist.append(efeat[:,0])

            # feat = normalize_features(feat)
            features_train_no_split.append(feat)
            edges_features_train_no_split.append(efeat)
            y_train_no_split.append(int(t[1][:-1]))

if method_number == 1:
  MEAN_COORDS /= len(features_train_no_split)
  print("MIN=",MIN_COORDS)
  print("MAX=",MAX_COORDS)
  print("MEAN=",MEAN_COORDS)
elif method_number == 2:
  all_coords = np.concatenate(all_coords)
  all_features = np.concatenate(all_features)
  all_dist = np.concatenate(all_dist)
  transformer.fit(all_coords)
  transformer_amino.fit(all_features)
  transformer_dist.fit(all_dist.reshape(-1,1))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.hist(all_dist, density=True, bins=30)  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');

In [ ]:
plt.hist(all_coords[:,0], density=True, bins=30)  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');
plt.show()

plt.hist(all_coords[:,1], density=True, bins=30)  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');
plt.show()

plt.hist(all_coords[:,2], density=True, bins=30)  # density=False would make counts
plt.ylabel('Probability')
plt.xlabel('Data');
plt.show()

In [ ]:
class EarlyStopping:
    def __init__(self, patience=1, min_delta=1, file_name="model.pt"):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
        self.file_name = file_name

    def early_stop(self, model, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
            print(f"Saving best model at: '{self.file_name}'")
            torch.save(model.state_dict(), self.file_name)
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

def normalize_features(feat,eps=1e-7):
  res = np.copy(feat)
  ##########
  #Method 1
  # res[:,0:3] -= MIN_COORDS
  # res[:,0:3] /= (MAX_COORDS-MIN_COORDS)
  # Method 2
  # res[:,0:3] -= MEAN_COORDS
  #Method 3
  # coords = res[:,0:3]
  # tmp_min,tmp_max = np.min(coords,axis=0),np.max(coords,axis=0)
  # res[:,0:3] -= tmp_min
  # res[:,0:3] /= (tmp_max-tmp_min + eps)
  #Method 4
  # coords = res[:,0:3]
  # tmp_mean = np.mean(coords,axis=0)
  # res[:,0:3] -= tmp_mean
  #Method 5
  res[:,0:3] = transformer.transform(np.clip(res[:,0:3],-250,250))
  res[:,25:] = transformer_amino.transform(res[:,25:])
  #################
  return res

def normalize_edges(efeat):
  res = np.copy(efeat)
  res[:,0] = transformer_dist.transform(np.clip(res[:,0],0,10).reshape(-1,1)).flatten()
  return res

def separate_edges(edge_index,edge_features,n=2):
  # print(edge_index)
  if n == 2:
    idx_bond = np.where(np.logical_or(edge_features[:,2],edge_features[:,4]))[0] #peptide or hydrogen
    idx_distance = np.where(np.logical_or(edge_features[:,1],edge_features[:,3]))[0] #distance

    return edge_index[:,idx_bond],edge_index[:,idx_distance]
  elif n== 4:
    idx_1 = np.where(edge_features[:,0])[0]
    idx_2 = np.where(edge_features[:,1])[0]
    idx_3 = np.where(edge_features[:,2])[0]
    idx_4 = np.where(edge_features[:,3])[0]

    return edge_index[:,idx_1],edge_index[:,idx_2],edge_index[:,idx_3],edge_index[:,idx_4]
  else:
    raise ValueError("Not implemented")
  

In [ ]:
from sklearn.model_selection import train_test_split
adj_train, adj_val, \
features_train, features_val, \
edges_features_train, edges_features_val,\
y_train, y_val = train_test_split(adj_train_no_split,features_train_no_split,
                                  edges_features_train_no_split, y_train_no_split, 
                                  test_size=0.2, random_state=42, stratify=y_train_no_split)
N_train = len(features_train)
N_val = len(features_val)

# Training

In [ ]:
# Hyperparameters
epochs = 200
batch_size = 64
n_input = 86
dropout = 0.5
learning_rate = 1e-3
n_class = 18
##########################
# Tunable
n_hidden = 64

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


NEW_NET = False
GRAPHS2 = True
GRAPHS4 = False
USE_EDGE_FEATURES = True


# model = GIN(n_input, n_hidden, dropout, n_class).to(device)
# with edge attributes
model = GIN2Graphs(n_input, n_hidden, dropout, n_class).to(device)

# model = GIN4Graphs(n_input, n_hidden, dropout, n_class).to(device)
# model = GINE(n_input, n_hidden, dropout, n_class).to(device)
# model = PDN(n_input, n_hidden, dropout, n_class).to(device)

#optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss(reduction='mean',label_smoothing=0.0)
model_name = f"2Graphs_nhidden{n_hidden}_normalized.pt"
early_stopping = EarlyStopping(patience=5, min_delta=0.1, file_name=model_name)



In [ ]:
def validation(model,normalize=False):
  model.eval()
  loss = 0
  correct = 0
  count = 0

  with torch.no_grad():
    for i in range(0, N_val, batch_size):
          adj_batch = list()
          features_batch = list()
          if USE_EDGE_FEATURES:
            edges_features_batch = list()
          idx_batch = list()
          y_batch = list()
          
          # Create tensors
          for j in range(i, min(N_val, i+batch_size)):
              n = adj_val[j].shape[0]
              adj_batch.append(adj_val[j]+sp.identity(n))
              ft_bt = features_val[j]
              if normalize:
                ft_bt = normalize_features(ft_bt)
              features_batch.append(ft_bt)
              if USE_EDGE_FEATURES:
                ed_ft = edges_features_val[j]
                if normalize:
                  ed_ft = normalize_edges(ed_ft)
                edges_features_batch.append(ed_ft)
              idx_batch.extend([j-i]*n)
              y_batch.append(y_val[j])
              
          adj_batch = sp.block_diag(adj_batch)
          features_batch = np.vstack(features_batch)
          if USE_EDGE_FEATURES:
            edges_features_batch = np.vstack(edges_features_batch)
          adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device).to_dense()
          # From https://discuss.pytorch.org/t/adjacency-matrix-to-edge-index-solution/148343
          # To get the edge_index representation from the adj matrix
          edge_index = adj_batch.nonzero().t().contiguous()

          if GRAPHS2:
            edge_index_bond, edge_index_distance = separate_edges(edge_index,edges_features_batch)
          elif GRAPHS4:
            e1,e2,e3,e4 = separate_edges(edge_index,edges_features_batch,n=4)
          
          if USE_EDGE_FEATURES and not (GRAPHS2 or GRAPHS4):
            edges_features_batch = torch.FloatTensor(edges_features_batch).to(device)
          features_batch = torch.FloatTensor(features_batch).to(device)
          idx_batch = torch.LongTensor(idx_batch).to(device)
          y_batch = torch.LongTensor(y_batch).to(device)

          if GRAPHS2:
            _, output = model(features_batch, edge_index_distance,edge_index_bond, idx_batch)
          elif GRAPHS4:
            _, output = model(features_batch,e1,e2,e3,e4, idx_batch)
          elif USE_EDGE_FEATURES:
            _, output = model(features_batch, edge_index, edges_features_batch, idx_batch)
          elif NEW_NET:
            output = model(features_batch, edge_index, idx_batch)
          else:
            _,output = model(features_batch, edge_index, idx_batch)
          loss += loss_function(output, y_batch).item()*output.size(0)
          count += output.size(0)
          preds = output.max(1)[1].type_as(y_batch)
          correct += torch.sum(preds.eq(y_batch).double())
  return loss/count,correct/count
        
      
        

def train(model,normalize=False):
  # Train model
  print(f"Training Model {model_name}")
  for epoch in range(epochs):
      t = time.time()
      train_loss = 0
      correct = 0
      count = 0
      # Iterate over the batches
      for i in range(0, N_train, batch_size):
          model.train()
          adj_batch = list()
          features_batch = list()
          if USE_EDGE_FEATURES:
            edges_features_batch = list()
          idx_batch = list()
          y_batch = list()
          
          # Create tensors

          for j in range(i, min(N_train, i+batch_size)):
              n = adj_train[j].shape[0]
              adj_batch.append(adj_train[j]+sp.identity(n))
              ft_bt = features_train[j]
              if normalize:
                ft_bt = normalize_features(ft_bt)
              features_batch.append(ft_bt)
              if USE_EDGE_FEATURES:
                ed_ft = edges_features_train[j]
                if normalize:
                  ed_ft = normalize_edges(ed_ft)
                edges_features_batch.append(ed_ft)
              idx_batch.extend([j-i]*n)
              y_batch.append(y_train[j])
              
          adj_batch = sp.block_diag(adj_batch)
          features_batch = np.vstack(features_batch)
          if USE_EDGE_FEATURES:
            edges_features_batch = np.vstack(edges_features_batch)
          adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device).to_dense()
          # From https://discuss.pytorch.org/t/adjacency-matrix-to-edge-index-solution/148343
          # To get the edge_index representation from the adj matrix
          edge_index = adj_batch.nonzero().t().contiguous()

          if GRAPHS2:
            edge_index_bond, edge_index_distance = separate_edges(edge_index,edges_features_batch)
          elif GRAPHS4:
            e1,e2,e3,e4 = separate_edges(edge_index,edges_features_batch,n=4)

          if USE_EDGE_FEATURES and not (GRAPHS2 or GRAPHS4):
              edges_features_batch = torch.FloatTensor(edges_features_batch).to(device)
          # edge_index = torch.LongTensor(edge_index).to(device)
          features_batch = torch.FloatTensor(features_batch).to(device)
          idx_batch = torch.LongTensor(idx_batch).to(device)
          y_batch = torch.LongTensor(y_batch).to(device)
          
          optimizer.zero_grad()
          if GRAPHS2:
            _, output = model(features_batch, edge_index_distance,edge_index_bond, idx_batch)
          elif GRAPHS4:
            _, output = model(features_batch, e1,e2,e3,e4, idx_batch)
          elif USE_EDGE_FEATURES:
            _, output = model(features_batch, edge_index,edges_features_batch, idx_batch)
          elif NEW_NET:
            output = model(features_batch, edge_index, idx_batch)
          else:
            _,output = model(features_batch, edge_index, idx_batch)
          loss = loss_function(output, y_batch)
          train_loss += loss.item() * output.size(0)
          count += output.size(0)
          preds = output.max(1)[1].type_as(y_batch)
          correct += torch.sum(preds.eq(y_batch).double())
          loss.backward()
          optimizer.step()
      
      if False:
          print('Epoch: {:03d}'.format(epoch+1),
                'loss_train: {:.4f}'.format(train_loss / count),
                'acc_train: {:.4f}'.format(correct / count),
                'time: {:.4f}s'.format(time.time() - t))
      if epoch % 1 == 0:
        loss_val,acc_val = validation(model,normalize=normalize)
        print('Epoch: {:03d}'.format(epoch+1),
                'loss_train: {:.4f}'.format(train_loss / count),
                'acc_train: {:.4f}'.format(correct / count),
                'loss_val: {:.4f}'.format(loss_val),
                'acc_val: {:.4f}'.format(acc_val),
                'time: {:.4f}s'.format(time.time() - t))
        if early_stopping.early_stop(model, loss_val):
          break

In [ ]:
train(model,normalize=True)

In [ ]:
model.load_state_dict(torch.load(model_name))
print("Validation of model:",validation(model,normalize=True))

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate/10)

In [ ]:
model.load_state_dict(torch.load(model_name))
train(model,normalize=True)

In [ ]:
model.load_state_dict(torch.load(model_name))
print("Validation of model:",validation(model,normalize=True))

# Creating Submission

In [ ]:
N_test = len(adj_test)
# Evaluate model
model.eval()
y_pred_proba = list()
# Iterate over the batches
for i in range(0, N_test, batch_size):
    adj_batch = list()
    idx_batch = list()
    features_batch = list()
    y_batch = list()
    edges_features_batch = list()
    
    # Create tensors
    for j in range(i, min(N_test, i+batch_size)):
        n = adj_test[j].shape[0]
        adj_batch.append(adj_test[j]+sp.identity(n))
        features_batch.append(normalize_features(features_test[j]))
        idx_batch.extend([j-i]*n)
        if USE_EDGE_FEATURES:
          edges_features_batch.append(normalize_edges(edges_features_test[j]))
            
    if USE_EDGE_FEATURES and not GRAPHS2:
        edges_features_batch = torch.FloatTensor(edges_features_batch).to(device)
    adj_batch = sp.block_diag(adj_batch)
    features_batch = np.vstack(features_batch)
    if USE_EDGE_FEATURES:
      edges_features_batch = np.vstack(edges_features_batch)
    adj_batch = sparse_mx_to_torch_sparse_tensor(adj_batch).to(device).to_dense()
    # From https://discuss.pytorch.org/t/adjacency-matrix-to-edge-index-solution/148343
    # To get the edge_index representation from the adj matrix
    edge_index = adj_batch.nonzero().t().contiguous()
    if GRAPHS2:
      edge_index_bond, edge_index_distance = separate_edges(edge_index,edges_features_batch)

    features_batch = torch.FloatTensor(features_batch).to(device)
    idx_batch = torch.LongTensor(idx_batch).to(device)

    if GRAPHS2:
      _,output = model(features_batch, edge_index_distance,edge_index_bond, idx_batch)
    else:
      _,output = model(features_batch, edge_index, idx_batch)
    y_pred_proba.append(output)
    
y_pred_proba = torch.cat(y_pred_proba, dim=0)
y_pred_proba = torch.exp(y_pred_proba)
y_pred_proba = y_pred_proba.detach().cpu().numpy()

# Write predictions to a file
with open('sample_submission.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(18):
        lst.append('class'+str(i))
    lst.insert(0, "name")
    writer.writerow(lst)
    for i, protein in enumerate(proteins_test):
        lst = y_pred_proba[i,:].tolist()
        lst.insert(0, protein)
        writer.writerow(lst)

 # Trash

In [ ]:

# else:
#   edges, features, edge_features = load_data(new_edge_loading=NEW_EDGE_LOADING) 

#   # Split data into training and test sets
#   edges_train_no_split = list()
#   features_train_no_split = list()
#   edges_features_train_no_split = list()
#   y_train_no_split = list()
#   edges_test = list()
#   features_test = list()
#   edges_features_test = list()
#   proteins_test = list()
#   print("Loading Data")
#   with open(os.path.join(PATH,'graph_labels.txt'), 'r') as f:
#       for i,line in tqdm(enumerate(f)):
#           t = line.split(',')
#           if len(t[1][:-1]) == 0:
#               proteins_test.append(t[0])
#               edges_test.append(edges[i])
#               features_test.append(features[i])
#               edges_features_test.append(edge_features[i])
#           else:
#               edges_train_no_split.append(edges[i])
#               features_train_no_split.append(features[i])
#               edges_features_train_no_split.append(edge_features[i])
#               y_train_no_split.append(int(t[1][:-1]))

# # Initialize device
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# # Compute number of training and test samples
# N_train_no_split = len(features_train_no_split)
# N_test = len(features_test)

In [ ]:
# #split data
# split_percentage = 0.9
# N = N_train_no_split
# indexes = np.arange(N)
# np.random.shuffle(indexes)
# idx_train = indexes[:int(N*split_percentage)]
# idx_val = indexes[int(N*split_percentage):]

# adj_train = list()
# features_train = list()
# edges_features_train = list()
# y_train = list()

# adj_val = list()
# features_val = list()
# edges_features_val = list()
# y_val = list()

# for i in range(N):
#   if i in idx_train:
#     adj_train.append(adj_train_no_split[i])
#     features_train.append(features_train_no_split[i])
#     edges_features_train.append(edges_features_train_no_split[i])
#     y_train.append(y_train_no_split[i])
#   elif i in idx_val:
#     adj_val.append(adj_train_no_split[i])
#     features_val.append(features_train_no_split[i])
#     edges_features_val.append(edges_features_train_no_split[i])
#     y_val.append(y_train_no_split[i])
#   else:
#     raise ValueError(f"Index i={i} is not in the training or validation set.")


# N_train = len(adj_train)
# N_val = len(adj_val)

In [ ]:
# graph_indicator = np.loadtxt(os.path.join(PATH,"graph_indicator.txt"), dtype=np.int64)
# _,graph_size = np.unique(graph_indicator, return_counts=True)

# edges = np.loadtxt(os.path.join(PATH,"edgelist.txt"), dtype=np.int64, delimiter=",")
# edges_inv = np.vstack((edges[:,1], edges[:,0]))
# edges = np.vstack((edges, edges_inv.T))
# s = edges[:,0]*graph_indicator.size + edges[:,1]
# idx_sort = np.argsort(s)
# edges = edges[idx_sort,:]
# edges,idx_unique =  np.unique(edges, axis=0, return_index=True)

In [ ]:


# #TODO: Edge Type try RGATConv, FiLMConv,RGCNConv
# #TODO: FeaturesTry NNConv,GENConv,
# def load_data(): 
#     """
#     Function that loads graphs
#     """  
#     graph_indicator = np.loadtxt(os.path.join(PATH,"graph_indicator.txt"), dtype=np.int64)
#     _,graph_size = np.unique(graph_indicator, return_counts=True)
    
#     edges = np.loadtxt(os.path.join(PATH,"edgelist.txt"), dtype=np.int64, delimiter=",")
#     A = sp.csr_matrix((np.ones(edges.shape[0]), (edges[:,0], edges[:,1])), shape=(graph_indicator.size, graph_indicator.size))
#     A += A.T
    
#     x = np.loadtxt(os.path.join(PATH,"node_attributes.txt"), delimiter=",")
#     edge_attr = np.loadtxt(os.path.join(PATH,"edge_attributes.txt"), delimiter=",")
    
#     adj = []
#     features = []
#     edge_features = []
#     idx_n = 0
#     idx_m = 0
#     for i in tqdm(range(graph_size.size)):
#         n_edges = adj[i].nnz
#         adj.append(A[idx_n:idx_n+graph_size[i],idx_n:idx_n+graph_size[i]])
#         edge_features.append(edge_attr[idx_m:idx_m+n_edges,:])
#         features.append(x[idx_n:idx_n+graph_size[i],:])
#         idx_n += graph_size[i]
#         idx_m += n_edges

#     return adj, features, edge_features

# def normalize_adjacency(A):
#     """
#     Function that normalizes an adjacency matrix
#     """
#     n = A.shape[0]
#     A = A + sp.identity(n)
#     degs = A.dot(np.ones(n))
#     inv_degs = np.power(degs, -1)
#     D = sp.diags(inv_degs)
#     A_normalized = D.dot(A)

#     return A_normalized

# def sparse_mx_to_torch_sparse_tensor(sparse_mx):
#     """
#     Function that converts a Scipy sparse matrix to a sparse Torch tensor
#     """
#     sparse_mx = sparse_mx.tocoo().astype(np.float32)
#     indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
#     values = torch.from_numpy(sparse_mx.data)
#     shape = torch.Size(sparse_mx.shape)
#     return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
# if new_edge_loading:
    #   edges = edges.T
    # if new_edge_loading:
    #   adj = []
    #   edges_index = []
    #   features = []
    #   edge_features = []
    #   idx_n = 0
    #   idx_m = 0
    #   for i in range(graph_size.size):
    #       adjecency_matrix = A[idx_n:idx_n+graph_size[i],idx_n:idx_n+graph_size[i]]
    #       adj.append(adjecency_matrix)
    #       #fixing edge representatoin
    #       # e = edges[:,idx_m:idx_m+adj[i].nnz]
    #       # e -= np.min(e)
    #       edge_index,_ = from_scipy_sparse_matrix(adjecency_matrix)
          
    #       edges_index.append(edge_index)
    #       # assert np.min(edges_index[-1]) == 0 
    #       feat = edge_attr[idx_m:idx_m+adj[i].nnz,:]
    #       edge_features.append(feat)
    #       ##########
          
    #       features.append(x[idx_n:idx_n+graph_size[i],:])
    #       idx_n += graph_size[i]
    #       idx_m += adj[i].nnz

    #   return edges_index, features, edge_features

    # else:

In [ ]:
# for i in range(len(features_val)):
#   features_val[i] = normalize_features(features_val[i])
#   edges_features_val[i] = normalize_edges(edges_features_val[i])

# for i in range(len(features_train)):
#   features_train[i] = normalize_features(features_train[i])
#   edges_features_train[i] = normalize_edges(edges_features_train[i])